# MT Data-conditioned VAE

CVAE/VAE code trained on Line 1 Central Region, with Line 3 as validation and Line 2 as test 1 (eventually - next step is to get test set(s) finalized and included in code)

In [1]:
# pip install tensorflow

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import time
import gc
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import csv
import scipy as sp
from scipy import interpolate
# import scipy.interpolate

from cgnn import cvae_mt64_updated as vae

: 

In [ ]:
run = 's4'
if not os.path.exists(run):
    os.makedirs(run)

: 

# Set up survey

In [ ]:
def import_file(file):
    with open(file, 'r') as f:
        reader = csv.reader(f)
        data = list(reader)
    line = np.array(data, dtype=float)
    cond = line[:,25:55]
    # obs_xs = line[:,270:285]
    # print(obs_xs.shape)
    obs_zs = -line[:,286:301]
    print(obs_zs.shape)
    print(cond.shape)
    # obs_total = np.concatenate((obs_xs, obs_zs), axis=1)
    # print(obs_total.shape)
    pred_zs = line[:,348:363]
    return line,cond, obs_zs, pred_zs

: 

In [ ]:
line1,cond1, obs1, pred1 = import_file('Line1_Central-Copy1.csv')
line2,cond2, obs2, pred2 = import_file('Line2_Central-Copy1.csv')
line3,cond3, obs3, pred3 = import_file('Line3_Central-Copy1.csv')



# print(obs1)

: 

In [ ]:
line94, cond94, obs94, pred94 = import_file('Central_Line94.csv')

line15, cond15, obs15, pred15 = import_file('Central_Line15.csv')
line30, cond30, obs30, pred30 = import_file('Central_Line30.csv')
line45, cond45, obs45, pred45 = import_file('Central_Line45.csv')
line60, cond60, obs60, pred60 = import_file('Central_Line60.csv')
line75, cond75, obs75, pred75 = import_file('Central_Line75.csv')
print(cond15)

: 

In [ ]:
# np.save('Line94_Central.npy', line94)
# np.save('Line94_Central_Conductivity.npy',cond94)
# np.save('Line94_Central_Observations.npy',obs94)

: 

In [ ]:
# remote reference frequencies
#   7.680002e+02   5.120000e+02   3.840000e+02   2.560000e+02   1.920000e+02   1.280000e+02
#   9.599997e+01   6.400000e+01   4.800001e+01   3.200000e+01   2.400000e+01   1.600000e+01
#   1.200000e+01   8.000000e+00   5.999999e+00   4.000000e+00   3.000000e+00   2.000000e+00
#   1.500000e+00   1.000000e+00   7.500002e-01   5.000000e-01   3.750000e-01   2.500000e-01
#   1.875000e-01


# mesh
# 64 ft
cell_size = 32*0.3048
depth_to_top = 32*0.3048
n_cells = 32
# one fewer depth; last cell extends to inf
#depths = depth_to_top + np.arange(1,n_cells)*cell_size
#depths to bottom
depths0 = np.array([4,8.4,13.24,18.56,24.42,30.86,37.95,45.74,54.31,63.74,74.11,85.52,98.07,111.88,127.07,143.78,162.16,182.38,204.62,229.08,255.99,285.59,318.15,353.97,393.37,436.71,484.38,536.82,594.5])
n_depths = len(depths0)
print(n_depths)
# data frequencies
# conservative, lines up with remote referenced stations, minus one frequency to avoid extrapolation
f_a = np.logspace(-2, 9, num=12, base=2)
f_b = np.logspace(-4, 7, num=12, base=2)*3
time0 = np.array([0.013,0.039,0.065,0.104,0.169,0.273,0.443,0.703,1.094,1.693,2.63,4.102,6.406,9.961,16.055])*1e-3
# frequencies = np.logspace(-4, 10, 15, base=2)
nt = len(time0)
print(nt)
# data are only real
ndata = nt

: 

In [ ]:
depth = np.append(0,depths0)

def format_models(file, depths):
    cond_new = np.zeros((len(file[:,0]),32))
    for i in range(len(file[:,0])):
        x = file[i]
        y = depths
        f = sp.interpolate.interp1d(y,x)
        y_new = [554,573]
        x_new = f(y_new)
        cond_new[i] = np.append(x,x_new)
    return cond_new

cond94_new = format_models(cond94,depth)

print(cond94_new.shape)
print(len(depth))  
# print(len(cond1_new[0]))
# print(cond1_new[1])


: 

In [ ]:
def format_raw(file,times):
    obs_new = np.zeros((len(file[:,0]),16))
    for i in range(len(file[:,0])):
        x = file[i]
        y = times
        f = sp.interpolate.interp1d(y,x)
        y_new = [16.054*1e-3]
        x_new = f(y_new)
        obs_new[i] = np.append(x,x_new)
    return obs_new
    
obs94_new = format_raw(obs94,time0)

print(obs94_new.shape)
print(len(time0))

: 

In [ ]:
from scipy.spatial.distance import cosine

# for i in range(0,1000,10):
#     print(np.mean(cond1[i]-cond1[0]))
#     # plt.plot(cond1[0],depth)
#     # plt.plot(cond1[i],depth)
#     # plt.gca().invert_yaxis()
def compute_moving_average(models, window_size=5):
    """
    Apply moving average to each model (row) in the dataset.
    """
    return np.array([
        np.convolve(model, np.ones(window_size)/window_size, mode='same')
        for model in models
    ])

def select_diverse_models(models, threshold=0.01, metric='mse', window_size=5):
    """
    Select a diverse subset of models based on moving average smoothed values.

    Parameters:
    - models: np.ndarray of shape (N, 30)
    - threshold: float, minimum difference to count as diverse
    - metric: str, 'mse' or 'cosine'
    - window_size: int, size of the moving average window

    Returns:
    - indices: np.ndarray of selected row indices
    """
    smoothed = compute_moving_average(models, window_size)
    selected = [0]  # Always include the first model

    for i in range(1, len(smoothed)):
        prev_model = smoothed[selected[-1]]
        curr_model = smoothed[i]

        if metric == 'mse':
            diff = np.mean((curr_model - prev_model)**2)
        elif metric == 'cosine':
            diff = cosine(curr_model, prev_model)
        else:
            raise ValueError("Unsupported metric. Use 'mse' or 'cosine'.")

        if diff > threshold:
            selected.append(i)

    return np.array(selected)

: 

In [ ]:
print(cond1.shape)

: 

In [ ]:
# Select diverse subset
diverse_indices1 = select_diverse_models(cond1, threshold=0.01, metric='mse', window_size=3)

# Extract selected models
diverse_models1 = cond1[diverse_indices1]
diverse_data1 = obs1[diverse_indices1]
print(len(diverse_models1), len(diverse_data1))

print(f"Selected {len(diverse_indices1)} diverse models from {len(cond1)} total")
print(diverse_models1.shape)

# Select diverse subset
diverse_indices15 = select_diverse_models(cond15, threshold=0.01, metric='mse', window_size=3)

# Extract selected models
diverse_models15 = cond15[diverse_indices15]
diverse_data15 = obs15[diverse_indices15]
print(len(diverse_models15), len(diverse_data15))

print(f"Selected {len(diverse_indices15)} diverse models from {len(cond15)} total")
print(diverse_models15.shape)

# Select diverse subset
diverse_indices30 = select_diverse_models(cond30, threshold=0.01, metric='mse', window_size=3)

# Extract selected models
diverse_models30 = cond30[diverse_indices30]
diverse_data30 = obs30[diverse_indices30]
print(len(diverse_models30), len(diverse_data30))

print(f"Selected {len(diverse_indices30)} diverse models from {len(cond30)} total")

# Select diverse subset
diverse_indices45 = select_diverse_models(cond45, threshold=0.01, metric='mse', window_size=3)

# Extract selected models
diverse_models45 = cond45[diverse_indices45]
diverse_data45 = obs45[diverse_indices45]
print(len(diverse_models45), len(diverse_data45))

print(f"Selected {len(diverse_indices45)} diverse models from {len(cond45)} total")

# Select diverse subset
diverse_indices60 = select_diverse_models(cond60, threshold=0.01, metric='mse', window_size=3)

# Extract selected models
diverse_models60 = cond60[diverse_indices60]
diverse_data60 = obs60[diverse_indices60]
print(len(diverse_models60), len(diverse_data60))

print(f"Selected {len(diverse_indices60)} diverse models from {len(cond60)} total")

# Select diverse subset
diverse_indices75 = select_diverse_models(cond75, threshold=0.01, metric='mse', window_size=3)

# Extract selected models
diverse_models75 = cond75[diverse_indices75]
diverse_data75 = obs75[diverse_indices75]
print(len(diverse_models75), len(diverse_data75))

print(f"Selected {len(diverse_indices75)} diverse models from {len(cond75)} total")

: 

In [ ]:
cond1_sampled = format_models(diverse_models1,depth)
obs1_sampled = format_raw(diverse_data1, time0)

cond15_new = format_models(diverse_models15,depth)
cond30_new = format_models(diverse_models30,depth)
cond45_new = format_models(diverse_models45,depth)
cond60_new = format_models(diverse_models60,depth)
cond75_new = format_models(diverse_models75,depth)

obs15_new = format_raw(diverse_data15,time0)
obs30_new = format_raw(diverse_data30,time0)
obs45_new = format_raw(diverse_data45,time0)
obs60_new = format_raw(diverse_data60,time0)
obs75_new = format_raw(diverse_data75,time0)

print(cond1_sampled.shape)
print(obs1_sampled.shape)
print(cond75_new.shape)
print(obs75_new.shape)

: 

In [ ]:
cond_sampled = np.append(cond1_sampled,cond15_new,axis=0)
cond_sampled = np.append(cond_sampled,cond30_new,axis=0)
cond_sampled = np.append(cond_sampled,cond45_new,axis=0)
cond_sampled = np.append(cond_sampled,cond60_new,axis=0)
cond_sampled = np.append(cond_sampled,cond75_new,axis=0)

obs_sampled = np.append(obs1_sampled, obs15_new, axis=0)
obs_sampled = np.append(obs_sampled, obs30_new, axis=0)
obs_sampled = np.append(obs_sampled, obs45_new, axis=0)
obs_sampled = np.append(obs_sampled, obs60_new, axis=0)
obs_sampled = np.append(obs_sampled, obs75_new, axis=0)

print(cond_sampled.shape)
print(obs_sampled.shape)

: 

In [ ]:
# --- Step 1: Load sampling pattern ---
# If your Excel file is actually saved as CSV, use this:
pattern = np.loadtxt('1D_ergodic_100_50.csv', delimiter=',', dtype=int)

# Flatten in case it’s multi-dimensional (e.g., 100×1 or 1×100)
pattern = pattern.flatten()
pattern_length = len(pattern)
print(f"Pattern length: {pattern_length}")

# --- Step 2: Load the large dataset (.npy file) ---
# data = np.load('large_dataset.npy')
model_length = len(cond_sampled)
data_length = len(obs_sampled)

print(f"Model length: {model_length}, Data length: {data_length}")

# --- Step 3: Repeat pattern to cover the full dataset ---
repeats = int(np.ceil(data_length / pattern_length))
repeated_pattern = np.tile(pattern, repeats)[:data_length]

# --- Step 4: Apply ergodic sampling mask ---
sampled_models = cond_sampled[repeated_pattern == 1]
sampled_data = obs_sampled[repeated_pattern == 1]

# --- Step 5: Save or return sampled data ---
# np.save('ergodic_sampled_output.npy', sampled_data)

print(f"Sampled {len(sampled_models)} and {len(sampled_data)} of {model_length} and {data_length} total points "
      f"({len(sampled_models) / model_length:.1%})."
      f"({len(sampled_data) / data_length:.1%})")

print(sampled_models.shape)

: 

In [ ]:
np.save('Sampled_Conductivity.npy',sampled_models)
np.save('Sampled_Observations.npy',sampled_data)

: 

In [ ]:
depths=np.append(depths0,[554,573])
print(depths.shape)

# np.save('Line94_Central_Conductivity.npy',cond94_new)

: 

In [ ]:
times = np.append(time0,16.054*1e-3)
times.sort()
print(len(times))

# np.save('Line94_Central_Observations.npy',obs94_new)

: 

In [ ]:
# x = np.arange(depths.size)
# new_length = 63
# new_x = np.linspace(x.min(), x.max(), new_length)
# new_depths = sp.interpolate.interp1d(x, depths)(new_x)
# print(new_depths)
# print(len(new_depths))
# depths = new_depths

: 

# Load training models and Create neural network

In [ ]:
# normalize model parameters between -1 and 1
# remember, min resistivity is 0.01, max is 1e5
# Gaussian infill potentially allows for values outside this range, but not likely
min_model = np.log(1e-5)
max_model = np.log(1e5)
# pad by norm_pad, so that a bunch of values don't end up at -1
norm_pad = 0.1

# create network
network = vae.CVAE(depths,
                   min_model=min_model,
                   max_model=max_model,
                   times = times,
                   norm_pad=norm_pad,
                   data_std=0.1,
                   model_std=.01,
                   beta_vae=4,
                   model_loss_type='mae',
                   data_loss_type='mae'
                  )

: 

In [ ]:
# Due to ranging orders of magnitude - recommended to keep
def preprocess(filename):
    '''
    Read RILD values from npy file
    Convert to log conductivity
    Reshape to include channel dimension
    '''
    x = np.log(np.load(filename))
    x = x.reshape(-1, n_cells, 1)
    return x

x_train_log = preprocess('Sampled_Conductivity.npy')
print(x_train_log.shape)
x_validate_log = preprocess('Line3_Central_Conductivity.npy')
x_test_log = preprocess('Line2_Central_Conductivity.npy')
x_test_log94 = preprocess('Line94_Central_Conductivity.npy')
# x_test2_log = preprocess('KGS_RILD_64ft_test2.npy')
# print(x_train_log)

x_train_raw = np.load('Sampled_Observations.npy')
x_validate_raw = np.load('Line3_Central_Observations.npy')
x_test_raw = np.load('Line2_Central_Observations.npy')
print(x_test_raw.shape)
x_test_raw94 = np.load('Line94_Central_Observations.npy')
print(x_test_raw94.shape)
#print(x_train_raw_log)

# x_train_raw = network.model_to_tanhs(x_train_raw_log)
# x_validate_raw = network.model_to_tanhs(x_validate_raw_log)
# x_test1_raw = network.model_to_tanhs(x_test1_raw_log)

x_train = network.model_to_tanhs(x_train_log)
x_validate = network.model_to_tanhs(x_validate_log)
x_test = network.model_to_tanhs(x_test_log)
x_test94 = network.model_to_tanhs(x_test_log94)
# x_test2 = network.model_to_tanhs(x_test2_log)
# print(x_train)
# print(np.max(x_train))

AuEM_models1 = np.load('Sampled_Conductivity.npy')
AuEM_models2 = np.load('Line2_Central_Conductivity.npy')
AuEM_models3 = np.load('Line3_Central_Conductivity.npy')

: 

In [ ]:
print(x_train.shape)
print(x_validate.shape)
print(x_train_raw.shape)
print(x_validate_raw.shape)

x_train1 = x_train
# print(x_train1)
x_validate1 = x_validate[0:3000,]
x_train_raw1 = x_train_raw
x_validate_raw1 = x_validate_raw[0:3000,]
# x_test_raw1 = x_test_raw[0:1000,]
# print(x_train_raw1)
# plt.plot(x_train_raw1[0],np.append(0,depths))

: 

In [ ]:
# compute stds
model_std = np.std(x_train.flatten())
#mt_data_file = 'KGS_MT.npy'
#os.remove(mt_data_file)
#try:
#    all_data = tf.convert_to_tensor(np.load(mt_data_file))
#except FileNotFoundError:
#    all_data = network.predict_tanh(x_train)
#    np.save(mt_data_file, all_data.numpy())
train_data = x_train1[:,:,0]
# print(train_data)
# print("train data", train_data.shape)
# print('.........')


raw_train = np.asarray(x_train_raw1.T)
print(np.shape(raw_train[:,0]))
print(times)
raw_train_T = raw_train*1e-15
raw_train_data = np.gradient(raw_train_T,times, axis=0)
raw_train_data = -np.abs(raw_train_data).T
# log_train_data = tf.math.log(-train_data)
#print('log_train_data'log_train_data.shape)
print("raw_train_data", raw_train_data.shape)
print(raw_train_data[0])
raw_validate = np.asarray(x_validate_raw1.T)
raw_validate_T = raw_validate*1e-15
raw_validate_data = np.gradient(raw_validate_T,times, axis=0)
raw_validate_data = -np.abs(raw_validate_data).T
validate_data = x_validate1[:,:,0]
# raw_validate_data = -x_validate_raw1

raw_test = np.asarray(x_test_raw.T)
raw_test_T = raw_test*1e-15
print(raw_test_T.shape)
raw_test_data = np.gradient(raw_test_T,times, axis=0)
raw_test_data = -np.abs(raw_test_data).T

raw_test94 = np.asarray(x_test_raw94.T)
raw_test94_T = raw_test94*1e-15
print(raw_test94_T.shape)
raw_test_data94 = np.gradient(raw_test94_T,times, axis=0)
raw_test_data94 = -np.abs(raw_test_data94).T

# print(raw_validate_data)
# test1_data = network.predict_tanh(x_test1)
# log_test1_data = tf.math.log(-test1_data)

# test2_data = network.predict_tanh(x_test2)
# log_test2_data = tf.math.log(-test2_data)

: 

In [ ]:
# plt.plot(train_data[0],np.append(0,depths))
# plt.plot(obs1_new[0], np.append(0,depths), color='red')

: 

In [ ]:
# np.save('train_data.npy',train_data)
# np.save('raw_train_data.npy',raw_train_data)
# np.save('validate_data.npy',validate_data)
# np.save('raw_validate_data.npy',raw_validate_data)

: 

In [ ]:
# train_data = np.load('train_data.npy')
# raw_train_data = np.load('raw_train_data.npy')
# validate_data = np.load('validate_data.npy')
# raw_validate_data = np.load('raw_validate_data.npy')
print(train_data.shape)
print(raw_train_data.shape)

# plt.plot(x_train1[0],np.append(0,depths))
# plt.plot(train_data[0],np.append(0,depths))

: 

In [ ]:
# # Create batches and shuffle
BATCH_SIZE = 50

#x_train1 = models *log transform models*; log_train_data: raw data

train_dataset = tf.data.Dataset.from_tensor_slices((
    tf.cast(train_data, tf.float32), 
    tf.cast(raw_train_data, tf.float32))).shuffle(10000).batch(BATCH_SIZE)

validate_dataset = tf.data.Dataset.from_tensor_slices((
    tf.cast(validate_data, tf.float32),
    tf.cast(raw_validate_data, tf.float32))).shuffle(10000).batch(x_validate1.shape[0])

# though the data vary over several orders of magnitude, 
# they don't vary so much within one frequency.
# data_std_vec = np.std(train_data.numpy(), axis=0)
# log_data_std_vec = np.std(log_train_data.numpy(), axis=0)
# log_data_std = np.std(log_train_data.numpy().flatten())
# average_log_data_std = np.std((log_train_data.numpy() - log_train_data.numpy().mean(axis=0)).flatten())
# print(model_std, log_data_std, average_log_data_std)
# compute elementwise stds
model_std_vec = np.std(x_train1, axis=0)
model_std_vec = np.reshape(model_std_vec, (n_cells))
# # compute relative std
mean_train_data = np.mean(train_data, axis=0)
rel_data_std = np.abs(mean_train_data)

# same for model, but mean over all
mean_model_value = np.mean(x_train1)
rel_model_std = 0.5*mean_model_value
# print(mean_model_value, rel_model_std)
print('data_std',rel_data_std)

: 

In [ ]:
# plt.plot(train_dataset[0],np.append(0,depths))

: 

In [ ]:
# finalize network
network = vae.CVAE(depths,
                   min_model=min_model,
                   max_model=max_model,
                   times=times,
                   norm_pad=norm_pad,
                   data_std=rel_data_std.reshape(1, -1, 1),
#                    model_std=model_std,
                   model_std=model_std_vec.reshape(1, -1, 1),
                   latent_dim=20,
                   beta_vae=0.01,
                   model_loss_type='mae',
                   data_loss_type='mae'
                  )

# double check forward model
'''

all_data[0, 0], all_data[0, 24]

network.predict_tanh(x_train[0:1])

c60 = np.load('KSG_RILD_60ft.npy')
c60[0].shape

vae.forward_1_freq(c60[200], depths, frequencies[0])[0]

vae.forward_1_freq(c60[200], depths, frequencies[-1])[0]

all_data.numpy().max(), all_data.numpy().min()

all_data.shape

'''

#for i_freq in range(nf):
#    plt.hist(all_data.numpy()[:, i_freq].flatten(), bins=50)

: 

In [ ]:
# May be able to remove/adjust steps since I have data instead of models
i_random_train = np.arange(16)
# random_train = x_train[i_random_train].reshape((16, n_cells))
# pick some random training models
# i_random_train = np.random.randint(0, x_train1.shape[0], 16)
#i_random_train = np.arange(16)
random_train = x_train1[i_random_train].reshape((16, 32))
# print('i_random_train',i_random_train)
# print('random_train',random_train)
print(network.predict_tanh(random_train))
# predict their data
# print(n_cells)
random_data = network.predict_tanh(random_train.reshape(16, 32, 1))
# print('n_cells',n_cells)
random_log_data = tf.math.log(-random_data)
# print('random_log_data', random_log_data)
print('random_data',random_data)
# Save data and latent space inputs for plots
# print(network.latent_dim)
latent_input = tf.random.normal([16, network.latent_dim], seed=0, dtype=tf.float32)
print('latent_input',latent_input)
# print('n_data',network.n_data)
random_data1 = random_data[:,:16]
data_input = tf.reshape(random_data1,(16,network.n_time))
print('data_input',data_input)
zd_input = tf.concat((latent_input,data_input),axis=-1)
print('zd_input',zd_input)
# Why are there nan values????? The forward modelling produces (16,30) arrays which should fit perfectly into data_input
#

: 

In [ ]:
# plot a few random training models and their data
# vae.plot_complex(random_data, times=times, save2file=True, filename=run+'/training_MT_data.png')
vae.plot_logs(np.exp(network.tanhs_to_model(random_train)), depths=depths, save2file=True, 
              filename=run + '/training_models.png')

# Save starting plots
network.plot_models(save2file=True,folder=run,samples=zd_input.shape[0],
                    latent=zd_input,step=0)
# network.plot_data(save2file=True,folder=run,latent=zd_input,step=0)
# network.plot_residuals(save2file=True,folder=run,latent=zd_input,step=0)

plt.close('all')
plt.clf()
gc.collect()

: 

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.003, 0.5)

: 

# Train

In [ ]:
epochs = 50

: 

In [ ]:
validate_terms = []
train_terms = []
train_losses = []
ttest_losses = []
# print(len(train_dataset))
for epoch in range(1, epochs + 1):
    start_time = time.time()
#     print(train_dataset)
#     print(optimizer)
    for train_x in train_dataset:#.batch(BATCH_SIZE):
        # print(train_x)
        train_loss, train_term = vae.compute_apply_gradients(network, train_x, optimizer, 
                                    use_data_misfit=True)
        #train_losses.append(train_loss.numpy())
        train_terms.append([tt.numpy() for tt in train_term])
#         for test_x in test_dataset:
#             ttest_loss = vae.compute_losses(network, test_x)
#             terms = [loss(l).numpy() for l in ttest_loss]
#             ttest_losses.append(terms)
    end_time = time.time()

    # compute and save losses
    for val_x in validate_dataset:#.batch(BATCH_SIZE):
        val_loss, val_term = vae.compute_loss(network, val_x)
        #terms = [tf.reduce_mean(l).numpy() for l in losses]
        #loss(vae.compute_reconstruction_loss(network, test_x))
    #elbo = -loss.result()
    validate_terms.append([tt.numpy() for tt in val_term])
    print('Epoch: {}, Data misfit: {:.4}, '
          'Reconstruction: {:.4}, '
          'KL: {:.4}, '
          'Elapsed {:.4} s'.format(epoch, train_term[0], train_term[1], train_term[2],
                                #elbo,
                                end_time - start_time))
        
    if epoch % 1e3 == 0:
        network.plot_models(save2file=True,folder=run,samples=zd_input.shape[0],
                 latent=zd_input,step=epoch)
        network.plot_data(save2file=True,folder=run,latent=zd_input,step=epoch)
        network.plot_residuals(save2file=True,folder=run,latent=zd_input,step=epoch)
        plt.close('all')
        gc.collect()

network.inference_net.save(run+'/encoder.h5')
network.generative_net.save(run+'/decoder.h5')
#np.save(run+'/optimizer_weights.npy', optimizer.load_weights())
np.save(run+'/losses.npy', np.array(validate_terms))
np.save(run+'/train_losses.npy', np.array(train_terms))


: 

# Load networks

In [ ]:
network.inference_net = tf.keras.models.load_model(run+'/encoder.h5')
network.generative_net = tf.keras.models.load_model(run+'/decoder.h5')

: 

In [ ]:
loss_terms = np.load(run+'/losses.npy')

: 

In [ ]:
#optimizer_weights = np.load(run+'/optimizer_weights.npy', allow_pickle=True)

: 

https://stackoverflow.com/questions/49503748/save-and-load-model-optimizer-state to train more

# Plot loss over epochs

In [ ]:
plt.rcParams.update({'font.size': 18})

: 

In [ ]:
tt = np.array(train_terms)
batches_per_epoch = tf.data.experimental.cardinality(train_dataset).numpy()
num_batches = tt.shape[0]
plt.figure(figsize=(10,6))
plt.semilogy(np.arange(num_batches)/batches_per_epoch, tt)
plt.ylabel('Loss')
plt.xlabel('Training epoch')
plt.title("Traning Loss")
plt.legend(['Data Misfit','Reconstruction error', 'KL divergence'])
plt.savefig('training_loss_'+run+'_w.png')
plt.show()

: 

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(np.arange(epochs),validate_terms)
plt.yscale('log')
plt.ylabel('Loss')
plt.xlabel('Training epoch')
plt.title('Validation Loss')
plt.legend(['Data misfit', 'Reconstruction error', 'KL divergence'])
plt.savefig('validation_loss_'+run+'_w.png')
plt.show()

: 

# View some tests

In [ ]:
vae.plot_logs(np.exp(network.tanhs_to_model(x_validate[0:16])), depths=depths, step=16, save2file=False)

: 

In [ ]:
z_model = network.reparameterize(*network.encode(x_validate[0:16]))
z_data = raw_validate_data[0:16]
zmd = tf.concat((z_model,z_data),-1)
network.plot_models(latent=zmd)

: 

In [ ]:
depth_centers = (network.depths[1:] + network.depths[:-1])/2
plot_depths = np.r_[
    depth_centers[0] - (depth_centers[1] - depth_centers[0]),
    depth_centers,
    depth_centers[-1] + depth_centers[-1] - depth_centers[-2]
]

zmd_tanhs = network.decode(zmd, apply_tanh=True)
zmd_samples = zmd_tanhs.shape[0]
zmd_tanhs = np.reshape(zmd_tanhs, (zmd_samples, network.n_model))
zmd_logs = network.tanhs_to_model(zmd_tanhs)
# zmdlogs = np.exp(zmd_logs)

true_validate = network.tanhs_to_model(x_validate[0:16])

: 

In [ ]:
#Why does x2 on the predicted make it match better???
# fig = plt.figure(figsize=(20, 20))

# for i in range(16):
#     plt.subplot(4, 4, i + 1)
#     plt.plot(true_validate[i], plot_depths, drawstyle='steps-post', label='True')
#     plt.plot(zmd_logs[i][:-2], plot_depths[:-2], drawstyle='steps-post', label='Pred', linestyle='--')
#     # plt.xlabel('Conductivity (S/m)')
#     # plt.ylabel('Depth (m)')
#     plt.suptitle('True vs Predicted Conductivity Plots',fontsize = 30)
#     plt.xscale('log')
#     plt.gca().invert_yaxis()
#     if i == 0:
#         plt.legend()

# fig.text(0.5, 0.06, 'Conductivity (S/m)', ha='center', va='center', size=20)
# fig.text(0.03, 0.5, 'Depth (m)', ha='center', va='center',
#              rotation='vertical', size=20)
# fig.tight_layout()
# plt.show()

fig = plt.figure(figsize=(20, 20))
# samples = data.shape[0]
subplot_rows = 4
subplot_cols = 4
for i in range(16):
        ax = plt.subplot(subplot_rows, subplot_cols, i+1)
        ax.set_title('Sounding %d' %int(i+1), fontsize = 16)
        ax.semilogx(np.exp(true_validate[i]), plot_depths, drawstyle='steps-post', label='True')
        ax.semilogx(np.exp(zmd_logs[i][:-2]), plot_depths[:-2], drawstyle='steps-post', label='Pred', linestyle='--')
        plt.gca().invert_yaxis()
plt.legend(bbox_to_anchor=(1.02, 0.1), loc='upper left', borderaxespad=0)
fig.text(0.5, 0.06, 'Conductivity (S/m)', ha='center', va='center', size=20)
fig.text(0.03, 0.5, 'Depth (m)', ha='center', va='center',
             rotation='vertical', size=20)
plt.suptitle('True vs Predicted Conductivity Plots',fontsize = 30)
plt.tight_layout(rect=(0.05,0.08,1,1))
plt.show()

: 

In [ ]:
def minmax(x): return tf.reduce_min(x), tf.reduce_max(x)

: 

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
mean_true1 = np.mean(true_validate,axis=0)
mean_pred1 = np.mean(zmd_logs,axis=0)

mae1_log = mean_absolute_error(mean_true1,mean_pred1)
mse1_log = mean_squared_error(mean_true1, mean_pred1)

mean_true1_lin = np.exp(mean_true1)
mean_pred1_lin = np.exp(mean_pred1)

mae1_linear = mean_absolute_error(mean_true1_lin,mean_pred1_lin)
mse1_linear = mean_squared_error(mean_true1_lin, mean_pred1_lin)

mult_fact1 = np.exp(mae1_log)
percent_error1 = (mult_fact1-1)*100

print(mae1_log, mse1_log)
print(mae1_linear, mse1_linear)
print(mult_fact1, percent_error1)

: 

In [ ]:
tanhs = network.decode(zmd, apply_tanh=True)
minmax(tanhs)
print(tanhs)

: 

In [ ]:
minmax(x_test[0:16])

: 

In [ ]:
print(zmd.shape)
network.plot_data(latent=zmd)

# d_obs is from validation seta

: 

In [ ]:
network.plot_residuals(latent=zmd, weighted=False, step=15)

: 

# Randomize latent; does data still fit?

In [ ]:
vae.plot_logs(np.exp(network.tanhs_to_model(x_test[0   :16])), depths=depths)

: 

In [ ]:
print(x_test[0:1,:,0].shape)
print(z_data[0:1].shape)

: 

In [ ]:
data0 = tf.tile(z_data[0:1],[16,1])
# print(latent_input)
data0 = tf.cast(data0, tf.float32)
# print(data0)
zmd2 = tf.concat((latent_input,data0),-1)
# print(zmd2)

: 

In [ ]:
network.plot_models(latent=zmd2)

: 

In [ ]:
zmd2_tanhs = network.decode(zmd2, apply_tanh=True)
zmd2_samples = zmd2_tanhs.shape[0]
zmd2_tanhs = np.reshape(zmd2_tanhs, (zmd2_samples, network.n_model))
zmd2_logs = network.tanhs_to_model(zmd2_tanhs)
# zmd2 = (10**zmd2_logs)-1


true_test = tf.tile(x_test[0:1,:,0],[16,1])
true_test = tf.cast(true_test, tf.float32)
true_test1 = network.tanhs_to_model(true_test)
# true_test1 = (10**true_test1)-1
print(true_test1)

: 

In [ ]:
zmd2_logs.shape

: 

In [ ]:
# plt.figure(figsize=(20, 20))

# for i in range(16):
#     plt.subplot(4, 4, i + 1)
#     plt.plot(true_test1[i], plot_depths, drawstyle='steps-post', label='True')
#     plt.plot(np.exp(zmd2_logs[i][:-2]), plot_depths[:-2], drawstyle='steps-post', label='Pred', linestyle='--')
#     plt.xscale('log')
#     # plt.plot(zmd2_logs[i], plot_depths, label='Pred_logs', linestyle='--')
#     plt.gca().invert_yaxis()
#     if i == 0:
#         plt.legend()
# plt.tight_layout()
# plt.show()

fig = plt.figure(figsize=(20, 20))
# samples = data.shape[0]
subplot_rows = 4
subplot_cols = 4
for i in range(16):
        ax = plt.subplot(subplot_rows, subplot_cols, i+1)
        ax.set_title('Sounding %d' %int(i+1), fontsize = 16)
        ax.semilogx(np.exp(true_test1[i]), plot_depths, drawstyle='steps-post', label='True')
        ax.semilogx(np.exp(zmd2_logs[i][:-2]), plot_depths[:-2], drawstyle='steps-post', label='Pred', linestyle='--')
        plt.gca().invert_yaxis()
plt.legend(bbox_to_anchor=(1.02, 0.1), loc='upper left', borderaxespad=0)
fig.text(0.5, 0.06, 'Conductivity (S/m)', ha='center', va='center', size=20)
fig.text(0.03, 0.5, 'Depth (m)', ha='center', va='center',
             rotation='vertical', size=20)
plt.suptitle('True vs Predicted Conductivity Plots',fontsize = 30)
plt.tight_layout(rect=(0.05,0.08,1,1))
plt.show()

: 

In [ ]:
mean_cond2 = np.mean(zmd2_logs,axis=0)
mean_true_cond2 = np.mean(true_test1, axis=0)
std_cond = np.std(zmd2_logs,axis=0)
print(std_cond)
print(mean_cond2)
mae2_log = mean_absolute_error(mean_true_cond2,mean_cond2)
mse2_log = mean_squared_error(mean_true_cond2, mean_cond2)

mean_true2_lin = np.exp(mean_true_cond2)
mean_pred2_lin = np.exp(mean_cond2)

mae2_linear = mean_absolute_error(mean_true2_lin,mean_pred2_lin)
mse2_linear = mean_squared_error(mean_true2_lin, mean_pred2_lin)

mult_fact2 = np.exp(mae2_log)
percent_error2 = (mult_fact2-1)*100

print(mae2_log, mse2_log)
print(mae2_linear, mse2_linear)
print(mult_fact2, percent_error2)

: 

In [ ]:
fig, ax = plt.subplots()
for log in zmd2_logs:
    ax.plot(np.exp(log[:-2]), plot_depths[:-2],drawstyle='steps-post', c='k', alpha=0.2)
ax.plot(np.exp(mean_cond2[:-2]), plot_depths[:-2], drawstyle='steps-post',label='Mean')
ax.plot(np.exp(mean_cond2[:-2]+std_cond[:-2]),plot_depths[:-2], drawstyle='steps-post', c='m', label="One Std")
ax.plot(np.exp(mean_cond2[:-2]-std_cond[:-2]), plot_depths[:-2], drawstyle='steps-post', c='m')
ax.plot(np.exp(mean_cond2[:-2]+2*std_cond[:-2]), plot_depths[:-2], drawstyle='steps-post', c='g', label= 'Two Std') # type: ignore
ax.plot(np.exp(mean_cond2[:-2]-2*std_cond[:-2]), plot_depths[:-2], drawstyle='steps-post', c='g')
# ax.plot(true_test1[1],plot_depths, c='orange', label = 'True Mean')
ax.invert_yaxis()
ax.axes.set_xlabel("Conductivity (S/m)")
ax.axes.set_ylabel("Depth (m)")
ax.axes.set_xscale('log')
# ax.axes.set_xlim(1e-1, 1e3)
ax.axes.set_title('(A)')
ax.axes.legend(fontsize=14)

plt.tight_layout()
plt.savefig('Preliminary_generated_plots_mean&2std.jpg')
plt.show()



: 

In [ ]:
fig, ax = plt.subplots()
for log in zmd2_logs:
    ax.semilogx(np.exp(log)[:-2], plot_depths[:-2],drawstyle='steps-post', c='k', alpha=0.2)
ax.semilogx(np.exp(mean_cond2)[:-2], plot_depths[:-2],drawstyle='steps-post',label='Mean')
ax.semilogx(np.exp(mean_cond2+std_cond**2)[:-2], plot_depths[:-2], drawstyle='steps-post', c='orange', label='Variance')
ax.semilogx(np.exp(mean_cond2-std_cond**2)[:-2], plot_depths[:-2], drawstyle='steps-post', c='orange')
ax.invert_yaxis()
ax.axes.set_xlabel("Conductivity (S/m)")
ax.axes.set_ylabel("Depth (m)")
# ax.axes.set_xlim(1e-4, 1e5)
ax.axes.set_title('(B)')
ax.axes.legend(fontsize=14)

plt.tight_layout()
plt.savefig('Preliminary_generated_plots.jpg')
plt.show()

: 

In [ ]:
# # Example inputs
# conductivities = [0.01, 0.1, 0.05, 0.2, 0.01]  # Conductivity (S/m) for each layer
# thicknesses = [10, 20, 20, 30, 20]  # Thickness of each layer (m)

# # Compute depths from thicknesses
# depths = np.cumsum(thicknesses)  # Compute layer boundaries
# depths = np.insert(depths, 0, 0)  # Insert surface depth at 0

# # Create step-like depth and conductivity arrays
# depth_plot = np.repeat(plot_depths, 2)  # Duplicate depths for step changes
# conductivity_plot = np.repeat(zmd2_logs[0], 2)  # Duplicate conductivity values

depth_plot = np.repeat(plot_depths, 2)
# depth_plot=np.flip(depth_plot)

fig, ax = plt.subplots(1,4,figsize=(20, 10))

# plt.ticklabel_format(axis='y', style='sci', scilimits=(4, 4))
# fig.suptitle('VAE Generated Models')
for log in zmd2:
    conductivity_plot = np.repeat(log,2)
    # ax[0].semilogx(conductivity_plot, depth_plot, drawstyle='steps-post', color='k', alpha=0.2)

# for i in AuEM_models3[0:1000,]:
    # ax[1].semilogx(i, plot_depths, drawstyle='steps-post', color='k', alpha=0.2)

mean_cond_plot = np.repeat(mean_cond2,2)
mean_true_cond_plot = np.repeat(mean_true_cond2,2)

difference = np.abs(np.exp(mean_true_cond2) + np.exp(mean_cond2))
print(np.mean(difference))
print(np.exp(mean_cond2))
print(mean_true_cond2)

ax[0].semilogx(np.exp(mean_cond2[:-2]),plot_depths[:-2], drawstyle='steps-post', c='b', label='Mean')
ax[0].invert_yaxis()
ax[0].set_xlabel('Conductivity (S/m)')
ax[0].set_ylabel('Depth (m)')
ax[0].set_title('Generated', fontsize=20)

ax[1].semilogx(np.exp(mean_true_cond2), plot_depths, drawstyle='steps-post', c='b', label='Mean')
ax[1].invert_yaxis()
ax[1].set_xlabel('Conductivity (S/m)')
ax[1].set_ylabel('Depth (m)')
ax[1].set_title('True', fontsize=20)

ax[2].semilogx(difference[:-2], plot_depths[:-2], drawstyle='steps-post', c='b')
ax[2].invert_yaxis()
ax[2].set_xlabel('Conductivity (S/m)')
ax[2].set_ylabel('Depth (m)')
ax[2].set_title('Difference', fontsize=20)

ax[3].semilogx(np.exp(mean_cond2[:-2]),plot_depths[:-2], drawstyle='steps-post', c='b', label='Pred')
ax[3].semilogx(np.exp(mean_true_cond2), plot_depths, drawstyle='steps-post', c='r', label='True')
ax[3].invert_yaxis()
ax[3].legend()
ax[3].set_xlabel('Conductivity (S/m)')
ax[3].set_ylabel('Depth (m)')
ax[3].set_title('Compared', fontsize=20)
# plt.grid()
plt.tight_layout()
plt.savefig('Comparison_GeneratedvsTrue.jpg')
plt.show()


: 

In [ ]:
-tf.exp(zmd2[0, 50:])

: 

In [ ]:
network.plot_data(latent=zmd2)

: 

In [ ]:
network.plot_residuals(latent=zmd2, weighted=True, step=14)

: 

In [ ]:
y_true = [[0., 0.], 
          [0., 0.]]
y_pred = [[3., 0.], 
          [2., 0.]]
# Using 'auto'/'sum_over_batch_size' reduction type.
# Using 'none' reduction type.
mse = tf.keras.losses.MeanSquaredError(
    reduction=tf.keras.losses.Reduction.NONE)
# Calling with 'sample_weight'.
mse(y_true, y_pred, sample_weight=[0.7, 0.3]).numpy()


: 

In [ ]:
1/2*((3-0)**2 + 0**2)*.7, 1/2*((2-0)**2 + (0-0)**2)*.3

: 

In [ ]:
network.data_weights

: 

# Sample latent & different data soundings

In [ ]:
# Plot true models for comparisons - need models for line 94 and adjust plot_logs to show true models too

data_batch = raw_test_data94[0:16]
latent_dim = 20  # known from model
latent_input = tf.random.normal((16, latent_dim))

: 

In [ ]:
zmd3 = tf.concat(((latent_input), data_batch), -1)
print(zmd3.shape)

: 

In [ ]:
zmd3_tanhs = network.decode(zmd3, apply_tanh=True)
zmd3_samples = zmd3_tanhs.shape[0]
zmd3_tanhs = np.reshape(zmd3_tanhs, (zmd3_samples, network.n_model))
zmd3_logs = network.tanhs_to_model(zmd3_tanhs)
# zmd3_exp = (10**zmd3_logs)

: 

In [ ]:
true_models = x_test94[0:2000]
true_logs = network.tanhs_to_model(true_models)
true_models = np.exp(true_logs)
print(true_logs[100])

: 

In [ ]:
vae.plot_logs(true_models[0:16], depths=depths)

: 

In [ ]:
network.plot_models(latent=zmd3)

: 

In [ ]:
# plt.figure(figsize=(20, 20))

# for i in range(16):
#     plt.subplot(4, 4, i + 1)
#     plt.plot(true_logs[i][:-2], plot_depths[:-2], drawstyle='steps-post', label='True')
#     plt.plot(np.exp(zmd3_logs[i])[:-2], plot_depths[:-2], drawstyle='steps-post', label='Pred', linestyle='--')
#     plt.xscale('log')
#     plt.gca().invert_yaxis()
#     if i == 0:
#         plt.legend()
# plt.tight_layout()
# plt.show()

fig = plt.figure(figsize=(20, 20))
# samples = data.shape[0]
subplot_rows = 4
subplot_cols = 4
for i in range(16):
        ax = plt.subplot(subplot_rows, subplot_cols, i+1)
        ax.set_title('Sounding %d' %int(i+1), fontsize = 16)
        ax.semilogx(np.exp(true_logs[i]), plot_depths, drawstyle='steps-post', label='True')
        ax.semilogx(np.exp(zmd3_logs[i][:-2]), plot_depths[:-2], drawstyle='steps-post', label='Pred', linestyle='--')
        plt.gca().invert_yaxis()
plt.legend(bbox_to_anchor=(1.02, 0.1), loc='upper left', borderaxespad=0)
fig.text(0.5, 0.06, 'Conductivity (S/m)', ha='center', va='center', size=20)
fig.text(0.03, 0.5, 'Depth (m)', ha='center', va='center',
             rotation='vertical', size=20)
plt.suptitle('True vs Predicted Conductivity Plots',fontsize = 30)
plt.tight_layout(rect=(0.05,0.08,1,1))
plt.show()

: 

In [ ]:
network.plot_data(latent=zmd3)
# Why only one sounding???

: 

In [ ]:
# zmd3_tanhs = network.decode(zmd3, apply_tanh=True)
# zmd3_samples = zmd3_tanhs.shape[0]
# zmd3_tanhs = np.reshape(zmd3_tanhs, (zmd3_samples, network.n_model))
# zmd3_logs = np.exp(network.tanhs_to_model(zmd3_tanhs))
# zmd3_exp = (10**zmd3_logs)

: 

In [ ]:
network.plot_residuals(latent=zmd3, step=13)

: 

In [ ]:
# plt.figure(figsize=(20, 20))

array1 = np.array(range(0,1600,100))
# print(array1)

# print(true_logs.shape)
# for i in range(len(array1)):
#     j = array1[i]
#     plt.subplot(4, 4,  i+1)
#     plt.plot(true_logs[j][:-2], plot_depths[:-2], drawstyle='steps-post')
#     plt.xscale('log')
#     plt.gca().invert_yaxis()
#     # if i == 0:
#     #     plt.legend()
# plt.tight_layout()
# plt.show()

fig = plt.figure(figsize=(20, 20))
# samples = data.shape[0]
subplot_rows = 4
subplot_cols = 4
for i in range(len(array1)):
        j = array1[i]
        ax = plt.subplot(subplot_rows, subplot_cols, i+1)
        ax.set_title('Sounding %d' %int(i+1), fontsize = 16)
        ax.semilogx(np.exp(true_logs[j][:-2]), plot_depths[:-2], drawstyle='steps-post', label='True')
        plt.gca().invert_yaxis()
# plt.legend(bbox_to_anchor=(1.02, 0.1), loc='upper left', borderaxespad=0)
fig.text(0.5, 0.06, 'Conductivity (S/m)', ha='center', va='center', size=20)
fig.text(0.03, 0.5, 'Depth (m)', ha='center', va='center',
             rotation='vertical', size=20)
plt.suptitle('Conductivity Plots',fontsize = 30)
plt.tight_layout(rect=(0.05,0.08,1,1))
plt.show()

: 

In [ ]:
mean_true3 = np.mean(true_logs,axis=0)
mean_pred3 = np.mean(zmd3_logs,axis=0)

mae3_log = mean_absolute_error(mean_true3,mean_pred3)
mse3_log = mean_squared_error(mean_true3, mean_pred3)

mean_true3_lin = np.exp(mean_true3)
mean_pred3_lin = np.exp(mean_pred3)

mae3_linear = mean_absolute_error(mean_true3_lin,mean_pred3_lin)
mse3_linear = mean_squared_error(mean_true3_lin, mean_pred3_lin)

mult_fact3 = np.exp(mae3_log)
percent_error3 = (mult_fact3-1)*100

print(mae3_log, mse3_log)
print(mae3_linear, mse3_linear)
print(mult_fact3, percent_error3)

: 

In [ ]:
lower_log = mean_pred3_lin/mult_fact3
upper_log = mean_pred3_lin*mult_fact3

lower_lin = np.clip(mean_pred3_lin-mae3_linear, a_min=1e-6, a_max=None)
upper_lin = mean_pred3_lin+mae3_linear

fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=True)

# Left: Linear error bands
axes[0].semilogx(mean_true3_lin, plot_depths, 'ko', label="Observed")
axes[0].semilogx(mean_pred3_lin[:-2], plot_depths[:-2], 'r-', label="Predicted")
axes[0].semilogx(np.log(lower_lin), plot_depths, 'b--', label="± Linear MAE")
axes[0].semilogx(np.log(upper_lin), plot_depths, 'b--')
axes[0].invert_yaxis()
axes[0].set_xlabel("Conductivity")
axes[0].set_title("Additive Error (Linear MAE)")
axes[0].legend()

# Right: Log error bands
axes[1].semilogx(mean_true3_lin, plot_depths, 'ko', label="Observed")
axes[1].semilogx(mean_pred3_lin[:-2], plot_depths[:-2], 'r-', label="Predicted")
axes[1].semilogx(lower_log, plot_depths, 'g--', label="×/÷ Factor Band")
axes[1].semilogx(upper_log, plot_depths, 'g--')
axes[1].invert_yaxis()
axes[1].set_xlabel("Conductivity")
axes[1].set_title("Multiplicative Error (Log MAE)")
axes[1].legend()

plt.gca().invert_yaxis()

plt.tight_layout()
plt.show()

: 

: 

: 